# Homework 5. Ранжирование и Гибридные рекоммендательные системы

### Подготовка

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from itertools import product

from metrics import precision_at_k
from utils import prefilter_items

/Users/nikolayzhukov/opt/anaconda3/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('data/retail_train.csv')

item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


## Filter items

In [5]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [6]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


# Prepare data set

In [7]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                       index='user_id', columns='item_id', 
                                       values='quantity', # Можно пробовать другие варианты
                                       aggfunc='count', 
                                       fill_value=0
                                      )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [9]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [10]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ


## Encoding features

In [12]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [13]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

## Подбор параметров модели

In [25]:
def metric_calc(data, result, model, metric):
    """Возвращает рассчитанную метрику"""
    # подготавливаемм id для юзеров и товаров в порядке пар user-item
    
    predictions = model.predict(user_ids=users_ids_row,
                                item_ids=items_ids_row,
                                user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                num_threads=10)
    
    data['score'] = predictions
    
    predict_result = data[['user_id','item_id','score']][data.item_id != 999999].drop_duplicates().\
    sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id'].unique().reset_index()
    
    # объединяем предикт и тест датасет для подсчета precision
    df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')
    
    res = df_result_for_metrics.apply(lambda row: metric(row['item_id'], row['actual'],k=5), axis=1).mean()
    
    return res

In [45]:
# параметры для подбора
parameters = {'no_components':[40, 60], 'learning_rate':[0.01, 0.05], 'item_alpha':[0.2, 0.5], 
              'user_alpha':[0.2, 0.5], 'max_sampled':[50, 100]}

In [46]:
# список с промежуточными результатами подбора
results_temp = []

In [47]:
%%time
# проходим циклом по всем возможным комбинациям параметров подбора
for comb in list(product(*parameters.values())):
    model = LightFM(no_components=comb[0],
                    loss='bpr',
                    learning_rate=comb[1], 
                    item_alpha=comb[2],
                    user_alpha=comb[3], 
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=comb[4])
    
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=10, 
              num_threads=20,
              verbose=True) 
    
    # сохраняем результаты расчета метрик
    results_temp.append({'no_components': comb[0], 
                         'learning_rate': comb[1],
                         'item_alpha': comb[2],
                         'user_alpha': comb[3],
                         'max_sampled': comb[4],
                         'precision@5': metric_calc(data_train_filtered, result, model, precision_at_k)})

Epoch: 100%|██████████| 10/10 [00:26<00:00,  2.61s/it]


CPU times: user 12min 3s, sys: 6.76 s, total: 12min 10s
Wall time: 12min 25s


In [48]:
results_temp_copy = pd.DataFrame(results_temp, 
                                 columns=['no_components', 'learning_rate', 'item_alpha',
                                          'user_alpha', 'max_sampled', 'precision@5'])
results_temp_copy.sort_values(by=['precision@5'], ascending=False).reset_index(drop=True)

,no_components,learning_rate,item_alpha,user_alpha,max_sampled,precision@5
0,60,0.01,0.2,0.2,50,0.143087
1,60,0.01,0.2,0.2,100,0.143087
2,40,0.01,0.2,0.5,50,0.142496
3,40,0.01,0.2,0.5,100,0.142496
4,40,0.01,0.2,0.2,100,0.142003
5,40,0.01,0.2,0.2,50,0.142003
6,60,0.01,0.2,0.5,100,0.140328
7,60,0.01,0.2,0.5,50,0.140328
8,60,0.05,0.5,0.2,100,0.136289
9,60,0.05,0.5,0.2,50,0.136289


Самые лучшие результаты получаются при низких значениях одновременно item_alpha и learning_rate, остальные параметры на результат влияют меньше.